In [ ]:
import os
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
%matplotlib inline
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
plt.rcParams['font.size'] = 7

from scipy.stats import ks_2samp
from sklearn.neighbors import KernelDensity

import pandas as pd


from context import src, utils
from src.analyzer import DataAnalyzer, plot_fill_between

output_dir = '../output/motion_benefit6'

In [ ]:
pkl_fns = [os.path.join(output_dir, fn) 
           for fn in os.listdir(output_dir) 
           if fn.endswith('.h5')]
pkl_fns.sort()
len(pkl_fns)

In [ ]:
da_ = [DataAnalyzer.fromfilename(pkl_fn) for pkl_fn in pkl_fns]

In [ ]:
for da in da_:
    da.s_range = 'pos'

In [ ]:
out_ = []
for da in da_:
# for pkl_fn in pkl_fns:
#     da = DataAnalyzer.fromfilename(pkl_fn)
    out = []
    out.append(da.data['EM_data']['mode']) # Eye path given or not
    out.append(da.data['motion_gen']['mode']) # True eye movements or no eye movements
    try:
        out.append(da.data['motion_gen']['dc'])
    except KeyError:
        out.append(0.0)
    out.append(da.data['motion_prior']['dc'])
    out.append(da.data['ds']) # Image size
    out = out + da.snr_list()
    out_.append(out)

In [ ]:
t = da.time_list()
data = pd.DataFrame.from_records(
    out_, columns=['inference_mode', 'gen_mode', 'gen_dc', 'prior_dc', 'ds'] + list(t))
grouped = pd.groupby(data, ['inference_mode', 'gen_mode', 'prior_dc', 'ds'])
c_ = plt.cm.rainbow(np.linspace(0, 1, len(grouped)))
np.random.shuffle(c_)
c_ = list(c_)
len(grouped)

In [ ]:
EXP = 'Experiment'
NOM = 'Diffusion'  # no motion

In [ ]:
for i, (name, group) in enumerate(grouped):
    mode, gen_mode, prior_dc, ds = name
#     if ds != 0.4:
#         continue
    print i, name
    print group[list(t)[-1]].mean()

In [ ]:
label_ = ['Motion', 'No-motion']

In [ ]:
plt.figure(figsize=(4, 4))
ax = plt.axes()
plt.title('SNR as a function of time')
alpha = 0.5
for c, (name, group) in zip(c_, grouped):
    inf_mode, gen_mode, prior_dc, ds = name
#     if ds != .40:
#         continue
#     label = ''
    label = 'Inf: {} Gen: {} GenDC:  InfDC: {} ds: {:.2f}'.format(*name)
    label = {EXP: 'Motion', NOM: 'No Motion'}[gen_mode]

    plot_fill_between(ax, t, group[list(t)], label=label, c=c, hatch=None, k=0.5)

#     plt.plot(t, group[list(t)].T.iloc[:, 0], c=c, label=label, alpha=alpha)
#     plt.plot(t, group[list(t)].T.iloc[:, 1:], c=c, alpha=alpha);

    plt.xlabel('time (ms)')
plt.legend(loc='upper left')
# plt.ylim([0, 12]);
# plt.savefig(os.path.join(output_dir, 'snr_time_no_motion_vs_motion_1.png'), dpi=200)

In [ ]:
plt.figure(figsize=(4, 4))
plt.title('SNR as a function of time')
alpha = 0.5
for c, (name, group) in zip(c_, grouped):
    inf_mode, gen_mode, prior_dc, ds = name
    label = {EXP: 'Motion', NOM: 'No Motion'}[gen_mode]
    plt.plot(t, group[list(t)].T.iloc[:, 0], c=c, label=label, alpha=alpha)
    plt.plot(t, group[list(t)].T.iloc[:, 1:], c=c, alpha=alpha);
    plt.xlabel('time (ms)')
plt.legend(loc='upper left')
# plt.savefig(os.path.join(output_dir, 'snr_time_no_motion_vs_motion_0.png'), dpi=200)

In [ ]:
plt.figure(figsize=(4, 4))
ax = plt.axes()
plt.title('SNR as a function of time')
alpha = 0.5
for c, (name, group) in zip(c_, grouped):
    inf_mode, gen_mode, prior_dc, ds = name
    label = {EXP: 'Motion', NOM: 'No Motion'}[gen_mode]
    plot_fill_between(ax, t, group[list(t)], label=label, c=c, k=0.5)
    plt.xlabel('time (ms)')
plt.legend(loc='upper left')
# plt.savefig(os.path.join(output_dir, 'snr_time_no_motion_vs_motion_1.png'), dpi=200)

In [ ]:
final_snrs = [group[list(t)[-1]].values for k, group in grouped]

res = ks_2samp(final_snrs[0], final_snrs[1])
label_ = [k[1] for k, group in grouped]

In [ ]:
res

In [ ]:
plt.figure(figsize=(4, 4))
X_plot = np.linspace(0, 20, 1000)[:, np.newaxis]
for c, final_snr, label in zip(c_, final_snrs, label_):
    log_dens = KernelDensity().fit(final_snr[:, np.newaxis]).score_samples(X_plot)
    plt.plot(X_plot, np.exp(log_dens), c=c, label=label)
plt.xlabel('SNR')
plt.ylabel('Normalized Density')
plt.title('Density of SNR, p = {:.1E}'.format(res.pvalue))
plt.legend()
plt.tight_layout()
# plt.savefig(os.path.join(output_dir, 'motion_benefit_density.png'), dpi=200)

In [ ]:
plt.hist(final_snrs, bins=20)

In [ ]:
idx = [data[(data['gen_mode'] == key) & (data['ds'] == 0.4)].index.values for key in [EXP, NOM]]
exp_idx, nom_idx = idx

In [ ]:
# np.random.shuffle(exp_idx)
# np.random.shuffle(nom_idx)

In [ ]:
def plot_results(ax, grouped):
    from matplotlib.patches import Rectangle
    c_ = ['b', 'g']
#     hatch_ = ['o', 'x']
    hatch_ = [None, None]
    alpha = 0.5
    label_ = []
    rect_ = []
    for c, hatch, (name, group) in zip(c_, hatch_, grouped):
        inf_mode, gen_mode, prior_dc, ds = name
        label = {EXP: 'Motion', NOM: 'No Motion'}[gen_mode]
        plot_fill_between(ax, t, group[list(t)], label=label, c=c, 
                          hatch=hatch, k=0.5)
        rect_.append(Rectangle((0, 0), 1, 1, fc=c, hatch=None))
        label_.append(label)

    ax.legend(rect_, label_, loc='upper left', prop={'size': 7})
    

# def equalize_y_axes(ax0, ax1):
#     u0, v0 = ax0.get_ylim()
#     u1, v1 = ax1.get_ylim()

#     u, v = min(u0, u1), max(v0, v1)
#     for ax in [ax0, ax1]:
#         ax.set_ylim(u, v)



def label_subplot(fig, ax, label, dx=0.005 * 4, dy=0.005):
    """
    Add a label to label a subplot offset from the plot.
    
    Parameters
    ----------
    fig : plt.Figure
        Overall figure to add label to.
    ax : plt.Axes()
        Axis corresponding to individual subplot.
    text : str
        Label
    dx, dy : float
        Amount to translate the letter relative to the top right corner of the subplot.
    """
    # Compose transformer:
    tf = ax.transAxes.inverted() + fig.transFigure
    # transFigure takes you from [0,1] figure coordinates to pixels
    # transAxes.inverted takes you from pixels to coordinates of axes
    # ax.text uses these coordinates
    
    # Transform delta
    u, v = tf.transform((dx, dy)) - tf.transform((0, 0))
    ax.text(-u, 1 + v, label, size=8, weight='bold', transform=ax.transAxes)


In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(3.5, 3.5))

da_exp, da_nom = [da_[idx_[4]] for idx_ in [exp_idx, nom_idx]]

ax = axes[0, 0]
da_exp.plot_image_and_rfs(fig, ax, -1)


ax = axes[0, 1]
plot_results(ax, grouped)
ax.set_xlim([0., 700.])
ax.set_xticks([0, 350, 700])
ax.set_title('SNR vs time (ms)')
# ax.set_xlabel('time (ms)')
# ax.set_ylabel('SNR')

# ax.xaxis.set_major_locator(MaxNLocator(3))
ax.yaxis.set_major_locator(MaxNLocator(4))

for v, da, label in zip(
    [0, 1],
    [da_exp, da_nom],
    ['Motion', 'No Motion']):
    ax = axes[1, v]
    da.plot_image_estimate(fig, ax, da.N_itr - 1,
                           colorbar=False)
    ax.set_title('{}: SNR = {:.2f}'.format(label, da.snr_one_iteration(da.N_itr - 1)))

    ax.xaxis.set_major_locator(MaxNLocator(3))
    ax.yaxis.set_major_locator(MaxNLocator(3))



for i, ax in enumerate(axes.flat):
    ax.set_title(ax.get_title(), fontdict={'size': 7})
    label=chr(ord('a') + i)
    label_subplot(fig=fig, ax=ax, label=label, dx=0.04, dy=0.01)

plt.tight_layout(pad=0.5)
plt.savefig(os.path.join(output_dir, 'motion_benefit.pdf'), dpi=200)

In [ ]:
da.plot_em_estimate(-1)
# plt.savefig(os.path.join(output_dir, 'ds04_e_reconstruction.png'), dpi=200)

In [ ]:
fig, ax = plt.subplots(figsize=(2, 2))
ax.set_xlabel(None, fontsize=7)
ax.set_ylabel(None, fontsize=7)
da.plot_image_and_rfs(fig, ax)
for tick in ax.xaxis.get_major_ticks():
    tick.label.set_fontsize(7)
for tick in ax.yaxis.get_major_ticks():
    tick.label.set_fontsize(7)
ax.title.set_fontsize(7)



plt.title('E Projected onto the Retina \n E thickness: {:.2f} arcmin \n Neuron Spacing {:.2f} arcmin'.format(
        2 * da.data['ds'], da.data['de']), fontsize=7)
a = 6; plt.xlim([-a, a]); plt.ylim([-a, a])


plt.tight_layout(pad=0.1)

# plt.savefig(os.path.join(output_dir, 'motion_benefit_e.pdf'))

In [ ]:
fig, axes = plt.subplots(5, 2, figsize=(4 * 2, 3.5 * 5))
for i, idx in enumerate(nom_idx[0:10]):
#     plt.subplot(5, 2, i + 1)
    da_[idx].plot_image_estimate(fig, axes.flat[i], -1)
plt.savefig(os.path.join(output_dir, 'e_reconstructions_no_motion.png'), dpi=200)

In [ ]:
fig, axes = plt.subplots(5, 2, figsize=(4 * 2, 3.5 * 5))
for i, idx in enumerate(exp_idx[0:10]):
    da_[idx].plot_image_estimate(fig, axes.flat[i], -1)
plt.savefig(os.path.join(output_dir, 'e_reconstructions_motion.png'), dpi=200)

In [ ]:
plt.figure(figsize=(20, 20))
for i, t in enumerate(range(0, 140, 10)):
    plt.subplot(5, 5, i + 1)
    da.plot_image_estimate(t)

# Group SNR plots by image size and diffusion constant

In [ ]:
out_ = []
for pkl_fn in pkl_fns:
    da = DataAnalyzer.fromfilename(pkl_fn)
    out = [da.data['ds'], da.data['motion_gen']['dc']]
    out = out + da.SNR_list()
    out_.append(out)

In [ ]:
t = da_[0].time_list()

In [ ]:
data = pd.DataFrame.from_records(out_, columns=['ds', 'dc_gen'] + list(t))
# data = data[data['ds'] == 0.5]

In [ ]:
grouped = pd.groupby(data, ['ds', 'dc_gen'])
c_ = ['g', 'b', 'r', 'y', 'm', 'c']

In [ ]:
plt.figure(figsize=(10, 7))
plt.title('SNR as a function of time')
for c, (name, group) in zip(c_, grouped):
    label = 'ds={}, dc={}'.format(*name)
    plot_fill_between(t, group[list(t)], label=label, c=c)
plt.legend(loc='upper left')
# plt.savefig(os.path.join(output_dir, 'motion_benefit.png'), dpi=200)